In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in /Users/mattiaskallman1/anaconda3/lib/python3.7/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

In [3]:
#os.getcwd()

In [4]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [5]:
settings = load_yaml('mock_settings.yaml')

In [6]:
#settings['ensemble']['models'][0]

In [7]:
dataframe = processing.create_dataframe(settings)

In [8]:
dataset = features.add(dataframe, settings['features'])

In [9]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [10]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [11]:
#settings['ensemble']['models'][1]['lstm']['layers']

In [12]:
# WORKING
#linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [13]:
# WORKING
#lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [14]:
# WORKING
#tcn = Models.train_model(data_lstm, 'tcn', settings['ensemble']['models'][3]['tcn'])

# [Time Based Cross Validation](https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8)
https://github.com/orhermansaffar/TimeBasedCV

In [25]:
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

class TimeBasedCV(object):
    '''
    Parameters 
    ----------
    train_period: int
        number of time units to include in each train set
        default is 30
    test_period: int
        number of time units to include in each test set
        default is 7
    freq: string
        frequency of input parameters. possible values are: days, months, years, weeks, hours, minutes, seconds
        possible values designed to be used by dateutil.relativedelta class
        deafault is days
    '''
    
    
    def __init__(self, train_period=30, test_period=7, freq='days'):
        self.train_period = train_period
        self.test_period = test_period
        self.freq = freq

        
        
    def split(self, data, validation_split_date=None, date_column='record_date', gap=0):
        '''
        Generate indices to split data into training and test set
        
        Parameters 
        ----------
        data: pandas DataFrame
            your data, contain one column for the record date 
        validation_split_date: datetime.date()
            first date to perform the splitting on.
            if not provided will set to be the minimum date in the data after the first training set
        date_column: string, deafult='record_date'
            date of each record
        gap: int, default=0
            for cases the test set does not come right after the train set,
            *gap* days are left between train and test sets
        
        Returns 
        -------
        train_index ,test_index: 
            list of tuples (train index, test index) similar to sklearn model selection
        '''
        
        # check that date_column exist in the data:
        try:
            data[date_column]
        except:
            raise KeyError(date_column)
                    
        train_indices_list = []
        test_indices_list = []

        if validation_split_date==None:
            validation_split_date = data[date_column].min().date() + eval('relativedelta('+self.freq+'=self.train_period)')
        
        start_train = validation_split_date - eval('relativedelta('+self.freq+'=self.train_period)')
        end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
        start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
        end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        while end_test < data[date_column].max().date():
            # train indices:
            cur_train_indices = list(data[(data[date_column].dt.date>=start_train) & 
                                     (data[date_column].dt.date<end_train)].index)

            # test indices:
            cur_test_indices = list(data[(data[date_column].dt.date>=start_test) &
                                    (data[date_column].dt.date<end_test)].index)
            
            #print("Train period:",start_train,"-" , end_train, ", Test period", start_test, "-", end_test,
            #      "# train records", len(cur_train_indices), ", # test records", len(cur_test_indices))

            train_indices_list.append(cur_train_indices)
            test_indices_list.append(cur_test_indices)

            # update dates:
            start_train = start_train + eval('relativedelta('+self.freq+'=self.test_period)')
            end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
            start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
            end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        # mimic sklearn output  
        index_output = [(train,test) for train,test in zip(train_indices_list,test_indices_list)]

        self.n_splits = len(index_output)
        
        return index_output
    
    
    def get_n_splits(self):
        """Returns the number of splitting iterations in the cross-validator
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits

In [16]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,0.788775,0.346918,0.120742,0.801102
2010-02-02,0.801102,0.382619,0.111121,0.783473
2010-02-03,0.783473,0.326071,0.081578,0.747343
2010-02-04,0.747343,0.310077,0.078393,0.732836
2010-02-05,0.732836,0.361426,0.085275,0.724710
...,...,...,...,...
2019-12-25,0.140181,0.592028,0.556347,0.141742
2019-12-26,0.141742,0.581530,0.512003,0.157833
2019-12-27,0.157833,0.596336,0.501223,0.160129


In [26]:
# How to use TimeBasedCV
data_for_modeling=dataset.reset_index()
tscv = TimeBasedCV(train_period=30,
                   test_period=7,
                   freq='days')
for train_index, test_index in tscv.split(data_for_modeling, date_column='Date_Timestamp'):
    continue

# get number of splits
tscv.get_n_splits()

512

In [27]:


#### Example- compute average test sets score: ####
X = data_for_modeling[['Close',columns]]
y = data_for_modeling[label]
from sklearn.linear_model import LinearRegression
import numpy as np

scores = []
for train_index, test_index in tscv.split(X, validation_split_date=datetime.date(2019,2,1)):

    data_train   = X.loc[train_index].drop('record_date', axis=1)
    target_train = y.loc[train_index]

    data_test    = X.loc[test_index].drop('record_date', axis=1)
    target_test  = y.loc[test_index]

    # if needed, do preprocessing here

    clf = LinearRegression()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # accuracy for the current fold only    
    r2score = clf.score(data_test,target_test)

    scores.append(r2score)

# this is the average accuracy over all folds
average_r2score = np.mean(scores)

NameError: name 'columns' is not defined

# [LightGBM](https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc)